https://mp.weixin.qq.com/s/KqZdITK-i8hSm4LO2F9B4Q

In [58]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [59]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [60]:
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [61]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('../../dataset/files/',train=True,download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize((0.1307,),(0.3081,))
                               ])
                              ),batch_size=batch_size_train,shuffle=True
)

In [62]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../../dataset/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [63]:
examples = enumerate(test_loader)

In [64]:
batch_idx,(example_data,example_targets) = next(examples)

In [65]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [66]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        print(x.shape)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        print(x.shape)
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [67]:
network = Net()
optimizer = optim.SGD(network.parameters(),lr=learning_rate,momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loadr.dataset) for i in range(n_epochs + 1)]

In [68]:
test_counter

[0, 60000, 120000, 180000]

In [69]:
def train(epoch):
    network.train()
    for batch_idx,(data,target) in enumerate(train_loadr):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch -1)*len(train_loadr.dataset)))
            torch.save(network.state_dict(),'../../dataset/model.pth')
            torch.save(optimizer.state_dict(),'../../dataset/optimizer.pth')

In [70]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output,target,size_average=False).item()
            pred = output.data.max(1,keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [71]:
test()
for epoch in range(1,n_epochs + 1):
    train(epoch)
    test()

torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])


C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])
torch.Size([1000, 10, 12, 12])
torch.Size([1000, 20, 4, 4])

Test set: Avg. loss: 2.3004, Accuracy: 751/10000 (7%)

torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.268457
torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
torch.Size([64, 10, 12, 12])
torch.Size([64, 20, 4, 4])
torch.Size([64, 10, 12,

In [73]:
# loading the model
continued_network = Net()
continued_optimizer = optim.SGD(network.parameters(),lr=learning_rate,momentum=momentum)

network_state_dict = torch.load('../../dataset/model.pth')
continued_network.load_state_dict(network_state_dict)

optimizer_state_dict = torch.load('../../dataset/optimizer.pth')
continued_optimizer.load_state_dict(optimizer_state_dict)

In [74]:
continued_network

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)